# Splunk App for Data Science and Deep Learning - Neural Network Classifier Notebook for TensorFlow

## Neural Network for Binary Classification
This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk. As an example we use a custom binary neural network classifier built on keras and tensorflow.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# mltkc_import
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

2024-02-05 17:04:12.292319: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)

numpy version: 1.25.2
pandas version: 2.1.1
TensorFlow version: 2.14.0
Keras version: 2.14.0


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup diabetes.csv <br>| fit MLTKContainer response from * algo=binary_nn_classifier epochs=10 mode=stage into app:diabetes_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("my_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [6]:
# mltkc_stage
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [7]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("diabetes_model")
print(df[0:1])
print(df.shape)
print(str(param))

   number_pregnant  glucose_concentration  blood_pressure  skin_thickness  \
0                6                    148              72              35   

   serum_insulin   BMI  diabetes_pedigree  age  response  
0              0  33.6              0.627   50         1  
(768, 9)
{'options': {'args': ['response', '*'], 'target_variable': ['response'], 'feature_variables': ['*'], 'params': {'algo': 'binary_nn_classifier', 'epochs': '10', 'mode': 'stage'}, 'model_name': 'diabetes_model', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '10000', 'max_distinct_cat_values_for_classifiers': '10000', 'max_distinct_cat_values_for_scoring': '10000', 'max_fit_time': '6000', 'max_inputs': '10000000', 'max_memory_usage_mb': '16000', 'max_model_size_mb': '3000', 'max_score_time': '6000', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['number_pregnant', 'glucose_concentration', 'blood_pressure', 'skin_thickness

## Stage 2 - create and initialize a model

In [8]:
# mltkc_init
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    X = df[param['feature_variables']]
    print("FIT build model with input shape " + str(X.shape))
    input_shape = int(X.shape[1])
    model_structure = '2-2'
    if 'options' in param:
        if 'params' in param['options']:
            if 'structure' in param['options']['params']:
                model_structure = str(param['options']['params']['structure']).lstrip("\"").rstrip("\"")
    hidden_factors = np.floor(np.array(model_structure.split("-"), dtype="float") * X.shape[1])
    model = keras.Sequential()
    model.add(keras.layers.Dense(input_shape, input_dim=input_shape, activation=tf.nn.relu))
    for hidden in hidden_factors:
        model.add(keras.layers.Dense(int(hidden), activation=tf.nn.relu))
    model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
# test mltkc_stage_create_model
model = init(df,param)
print(model.summary())

FIT build model with input shape (768, 8)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 505 (1.97 KB)
Trainable params: 505 (1.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Stage 3 - fit the model

In [10]:


# mltkc_stage_create_model_fit
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    X = df[param['feature_variables']]
    Y = df[param['target_variables']]
    model_epochs = 100
    model_batch_size = None
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model_epochs = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
    # connect model training to tensorboard
    log_dir="/srv/notebooks/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    # run the training
    returns['fit_history'] = model.fit(x=X,
                                       y=Y, 
                                       verbose=2, 
                                       epochs=model_epochs, 
                                       batch_size=model_batch_size, 
                                       #validation_data=(X, Y),
                                       callbacks=[tensorboard_callback])
    # memorize parameters
    returns['model_epochs'] = model_epochs
    returns['model_batch_size'] = model_batch_size
    returns['model_loss_acc'] = model.evaluate(x = X, y = Y)
    return returns

In [11]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

Epoch 1/10
24/24 - 1s - loss: 2.2804 - accuracy: 0.5260 - 920ms/epoch - 38ms/step
Epoch 2/10
24/24 - 0s - loss: 1.0764 - accuracy: 0.5156 - 113ms/epoch - 5ms/step
Epoch 3/10
24/24 - 0s - loss: 0.8037 - accuracy: 0.5794 - 129ms/epoch - 5ms/step
Epoch 4/10
24/24 - 0s - loss: 0.7149 - accuracy: 0.6289 - 115ms/epoch - 5ms/step
Epoch 5/10
24/24 - 0s - loss: 0.6784 - accuracy: 0.6719 - 97ms/epoch - 4ms/step
Epoch 6/10
24/24 - 0s - loss: 0.6764 - accuracy: 0.6549 - 88ms/epoch - 4ms/step
Epoch 7/10
24/24 - 0s - loss: 0.6590 - accuracy: 0.6732 - 91ms/epoch - 4ms/step
Epoch 8/10
24/24 - 0s - loss: 0.6432 - accuracy: 0.6888 - 95ms/epoch - 4ms/step
Epoch 9/10
24/24 - 0s - loss: 0.6340 - accuracy: 0.6823 - 89ms/epoch - 4ms/step
Epoch 10/10
24/24 - 0s - loss: 0.6244 - accuracy: 0.6940 - 86ms/epoch - 4ms/step
24/24 [==============================] - 0s 2ms/step - loss: 0.6172 - accuracy: 0.7018
[0.6172060966491699, 0.7018229365348816]


## Stage 4 - apply the model

In [12]:
# mltkc_stage_create_model_apply
def apply(model,df,param):
    X = df[param['feature_variables']]
    y_hat = model.predict(x = X, verbose=1)
    return y_hat

In [13]:
# test mltkc_stage_create_model_apply
y_hat = apply(model,df,param)
print(y_hat)

24/24 [==============================] - 0s 1ms/step
[[0.7114761 ]
 [0.21236587]
 [0.5777898 ]
 [0.4070474 ]
 [0.48207155]
 [0.39695954]
 [0.37825987]
 [0.5959869 ]
 [0.6062622 ]
 [0.17158228]
 [0.18499213]
 [0.81608135]
 [0.53511477]
 [0.64291495]
 [0.34789792]
 [0.6019654 ]
 [0.2673979 ]
 [0.29628313]
 [0.42409694]
 [0.3525491 ]
 [0.30227092]
 [0.09643285]
 [0.8465335 ]
 [0.33051452]
 [0.4082396 ]
 [0.34458447]
 [0.59522814]
 [0.34652722]
 [0.19392641]
 [0.16743204]
 [0.1841897 ]
 [0.47052112]
 [0.25551564]
 [0.2033445 ]
 [0.3628262 ]
 [0.34392956]
 [0.515683  ]
 [0.15230054]
 [0.14950508]
 [0.22765267]
 [0.6285673 ]
 [0.31451705]
 [0.14762643]
 [0.34509435]
 [0.59304893]
 [0.84423035]
 [0.7135037 ]
 [0.19017066]
 [0.29247227]
 [0.19841127]
 [0.29003525]
 [0.37067285]
 [0.21796373]
 [0.27131176]
 [0.38751337]
 [0.248974  ]
 [0.4791932 ]
 [0.16978379]
 [0.52579075]
 [0.39514372]
 [0.26610696]
 [0.563563  ]
 [0.10521518]
 [0.6108304 ]
 [0.51461035]
 [0.19325133]
 [0.15146965]
 [0.07361

## Stage 5 - save the model

In [14]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # save keras model
    model.save(MODEL_DIRECTORY + name + ".keras")
    return model

## Stage 6 - load the model

In [16]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    # save keras model
    model = keras.models.load_model(MODEL_DIRECTORY + name + ".keras")
    return model

## Stage 7 - provide a summary of the model

In [17]:
# return model summary
def summary(model=None):
    returns = {"version": {"tensorflow": tf.__version__, "keras": keras.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        model.summary(print_fn=lambda x: s.append(x+'\n'))
        returns["summary"] = ''.join(s)
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code

In [18]:
summary(model)

{'version': {'tensorflow': '2.14.0', 'keras': '2.14.0'},
 'summary': 'Model: "sequential"\n_________________________________________________________________\n Layer (type)                Output Shape              Param #   \n=================================================================\n dense (Dense)               (None, 8)                 72        \n                                                                 \n dense_1 (Dense)             (None, 16)                144       \n                                                                 \n dense_2 (Dense)             (None, 16)                272       \n                                                                 \n dense_3 (Dense)             (None, 1)                 17        \n                                                                 \n=================================================================\nTotal params: 505 (1.97 KB)\nTrainable params: 505 (1.97 KB)\nNon-trainable params: 0 (0.00 Byte)\n______